<a href="https://colab.research.google.com/github/alishahidklair/Chat_App/blob/main/Llama_3_1_%26_70B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install llama-toolchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 1.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.3/114.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.6 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.6.0-py2.py3-none-any.whl size=117030 sha256=e595b45609e0951175f4f98ba41753b1cffd9bb3e86e2abfbd5f7da4f96d087c
  Stored in directory: /root/.cache/pip/wheels/d6/6d/5d/5b73fa0f46d01a793713f8859201361e9e581ced8c75e5c6a3
Successfully built fire


In [2]:
!llama model list

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:161: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
+---------------------------------------+---------------------------------------------+----------------+----------------------------+
| Model Descriptor                      | HuggingFace Repo                            | Context Length | Hardware Requirements      |
+---------------------------------------+---------------------------------------------+----------------+----------------------------+
| Meta-Llama3.1-8B                      | meta-llama/Meta-Llama-3.1-8B                | 128K           | 1 GPU, each >= 20GB VRAM   |
+---------------------------------------+---------------------------------------------+----------------+----------------------------+
| Meta-Llama3.1-70B                     | meta-llama/Meta-Llama

In [8]:
!echo "url from Meta" | llama download --source meta --model-id Meta-Llama3.1-8B

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:161: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
Please provide the signed URL you received via email (e.g., https://llama3-1.llamameta.net/*?Policy...): Downloading `checklist.chk`...
Progress: |██████████████████████████████████████████████████| 100.00% (0/0 MB) Speed: 0.00 MiB/s
Finished downloading `/root/.llama/checkpoints/Meta-Llama3.1-8B/checklist.chk`....
Progress: |██████████████████████████████████████████████████| 100.00% (2/2 MB) Speed: 8.95 MiB/s
Finished downloading `/root/.llama/checkpoints/Meta-Llama3.1-8B/tokenizer.model`....
Progress: |██████████████████████████████████████████████████| 100.00% (0/0 MB) Speed: 0.00 MiB/s
Finished downloading `/root/.llama/checkpoints/Meta-Llama3.1-8B/params.json`....
Progress: |███████████████████████████████████████████

In [11]:
!pip install torch fairscale fire blobfile

In [15]:
!CHECKPOINT_DIR=~/.llama/checkpoints/Meta-Llama-3.1-8B

In [17]:
!PYTHONPATH=$(git rev-parse --show-toplevel) torchrun models/scripts/example_chat_completion.py $CHECKPOINT_DIR

fatal: not a git repository (or any of the parent directories): .git
/usr/bin/python3: can't open file '/content/models/scripts/example_chat_completion.py': [Errno 2] No such file or directory
E0823 16:00:17.097000 138941462761920 torch/distributed/elastic/multiprocessing/api.py:826] failed (exitcode: 2) local_rank: 0 (pid: 11615) of binary: /usr/bin/python3
Traceback (most recent call last):
  File "/usr/local/bin/torchrun", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/torch/distributed/elastic/multiprocessing/errors/__init__.py", line 347, in wrapper
    return f(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torch/distributed/run.py", line 879, in main
    run(args)
  File "/usr/local/lib/python3.10/dist-packages/torch/distributed/run.py", line 870, in run
    elastic_launch(
  File "/usr/local/lib/python3.10/dist-packages/torch/distributed/launcher/api.py", line 132, in __call__
    return launch_agent(self._config, self

In [1]:
import os
import torch
from transformers import LlamaForCausalLM, LlamaTokenizer

def test_model(checkpoint_dir):
    # Expand user directory (if necessary)
    checkpoint_dir = os.path.expanduser(checkpoint_dir)

    # Verify that the tokenizer model file exists
    tokenizer_file = os.path.join(checkpoint_dir, 'tokenizer.model')
    if not os.path.exists(tokenizer_file):
        raise FileNotFoundError(f"Tokenizer model file does not exist at {tokenizer_file}")

    try:
        # Load the tokenizer and model
        tokenizer = LlamaTokenizer.from_pretrained(checkpoint_dir, legacy=False)  # Use legacy=False if needed
        model = LlamaForCausalLM.from_pretrained(checkpoint_dir)
    except Exception as e:
        print(f"Error loading tokenizer or model: {e}")
        return

    # Move the model to GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Define a simple input prompt
    prompt = "Once upon a time in a land far away,"
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

    # Generate the completion
    try:
        with torch.no_grad():
            output = model.generate(input_ids, max_length=50, do_sample=True)
    except Exception as e:
        print(f"Error during model generation: {e}")
        return

    # Decode and print the output
    completion = tokenizer.decode(output[0], skip_special_tokens=True)
    print("Prompt:", prompt)
    print("Completion:", completion)

# Set your checkpoint directory
checkpoint_dir = '/root/.llama/checkpoints/Meta-Llama3.1-8B'

# Call the function with the checkpoint directory
test_model(checkpoint_dir)


Error loading tokenizer or model: Error parsing message with type 'sentencepiece.ModelProto'


In [37]:
import os
checkpoint_dir = '/root/.llama/checkpoints/Meta-Llama3.1-8B'

tokenizer_file = os.path.join(checkpoint_dir, 'tokenizer.model')
if os.path.exists(tokenizer_file):
    print("Tokenizer model file exists!")
else:
    print("Tokenizer model file does not exist.")


Tokenizer model file exists!


In [38]:
!pip install --upgrade sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 6.9 MB/s eta 0:00:00
  Attempting uninstall: sentencepiece
    Found existing installation: sentencepiece 0.1.99
    Uninstalling sentencepiece-0.1.99:
      Successfully uninstalled sentencepiece-0.1.99


In [2]:
!echo "url from Meta" | llama download --source meta --model-id Meta-Llama3.1-8B-Instruct

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:161: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
Please provide the signed URL you received via email (e.g., https://llama3-1.llamameta.net/*?Policy...): Downloading `checklist.chk`...
Progress: |██████████████████████████████████████████████████| 100.00% (0/0 MB) Speed: 0.00 MiB/s
Finished downloading `/root/.llama/checkpoints/Meta-Llama3.1-8B-Instruct/checklist.chk`....
Progress: |██████████████████████████████████████████████████| 100.00% (2/2 MB) Speed: 3.52 MiB/s
Finished downloading `/root/.llama/checkpoints/Meta-Llama3.1-8B-Instruct/tokenizer.model`....
Progress: |██████████████████████████████████████████████████| 100.00% (0/0 MB) Speed: 0.00 MiB/s
Finished downloading `/root/.llama/checkpoints/Meta-Llama3.1-8B-Instruct/params.json`....
Progress: |████████████████

In [3]:
!pip install transformers torch sentencepiece

In [10]:
import torch
from transformers import LlamaForCausalLM, LlamaTokenizer

def generate_response(model, tokenizer, prompt, max_length=150, temperature=0.7, top_k=50, top_p=0.95):
    # Encode the input prompt
    input_ids = tokenizer(prompt, return_tensors='pt').input_ids

    # Move tensors to the appropriate device (GPU or CPU)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    input_ids = input_ids.to(device)

    # Generate text with improved parameters
    with torch.no_grad():
        output = model.generate(
            input_ids,
            max_length=max_length,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            do_sample=True
        )

    # Decode the output
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response


# Define the checkpoint directory
checkpoint_dir = '/root/.llama/checkpoints/Meta-Llama3.1-8B-Instruct'

# Load the tokenizer and model

tokenizer = AutoTokenizer.from_pretrained('distilgpt2')
model = AutoModelForCausalLM.from_pretrained('distilgpt2')


# Define a prompt
prompt = "Explain the theory of relativity in simple terms."

# Generate and print the response
response = generate_response(model, tokenizer, prompt)
print("Prompt:", prompt)
print("Response:", response)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt: Explain the theory of relativity in simple terms.
Response: Explain the theory of relativity in simple terms. The first of these theories was the theory of relativity. It was first introduced by Einstein in a letter to the physicist in 1857. In this letter Einstein stated that the theory of relativity is not a theoretical theory of relativity. It was originally developed in the 1920s and as a result of a series of research by Einstein in the 1930s. The theory of relativity was formulated by Einstein in 1857, but it was not formally formulated until the 1930s. In the 1920s, in addition to the theory of relativity, the theory of relativity was expanded into the modern theory of relativity.







The first idea for the theory of relativity was the theory of relativity.


In [11]:
!torchrun --nproc_per_node 1 example_text_completion.py \
    --ckpt_dir Meta-Llama-3.1-8B/ \
    --tokenizer_path tokenizer.model \
    --max_seq_len 10 --max_batch_size 4

/usr/bin/python3: can't open file '/content/example_text_completion.py': [Errno 2] No such file or directory
E0823 16:31:42.296000 139913511125440 torch/distributed/elastic/multiprocessing/api.py:826] failed (exitcode: 2) local_rank: 0 (pid: 19643) of binary: /usr/bin/python3
Traceback (most recent call last):
  File "/usr/local/bin/torchrun", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/torch/distributed/elastic/multiprocessing/errors/__init__.py", line 347, in wrapper
    return f(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torch/distributed/run.py", line 879, in main
    run(args)
  File "/usr/local/lib/python3.10/dist-packages/torch/distributed/run.py", line 870, in run
    elastic_launch(
  File "/usr/local/lib/python3.10/dist-packages/torch/distributed/launcher/api.py", line 132, in __call__
    return launch_agent(self._config, self._entrypoint, list(args))
  File "/usr/local/lib/python3.10/dist-packages/torch/dist

In [14]:
!ls /path/to/models/

ls: cannot access '/path/to/models/': No such file or directory


In [17]:
!ls /root/.llama/checkpoints/Meta-Llama3.1-8B-Instruct/

Meta-Llama3.1-405B:bf16-mp16  Meta-Llama3.1-8B	Meta-Llama3.1-8B-Instruct


In [2]:
from transformers import LlamaTokenizer, LlamaForCausalLM
import torch

# Path to the model directory
model_path = '/root/.llama/checkpoints/Meta-Llama3.1-8B-Instruct/'

# Load the tokenizer and model
tokenizer = LlamaTokenizer.from_pretrained(model_path, legacy=False)
model = LlamaForCausalLM.from_pretrained(model_path)

# Example usage
inputs = tokenizer("Hello, how are you?", return_tensors="pt")
outputs = model.generate(**inputs)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


DecodeError: Error parsing message with type 'sentencepiece.ModelProto'

In [19]:
!pip install --upgrade transformers sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 621.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 29.5 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.4
    Uninstalling transformers-4.42.4:
      Successfully uninstalled transformers-4.42.4


In [8]:
# Copyright (c) Meta Platforms, Inc. and affiliates.
# This software may be used and distributed according to the terms of the Llama 2 Community License Agreement.

from typing import List, Optional

import fire

from llama import Llama, Dialog


def main(
    ckpt_dir: str,
    tokenizer_path: str,
    temperature: float = 0.6,
    top_p: float = 0.9,
    max_seq_len: int = 512,
    max_batch_size: int = 8,
    max_gen_len: Optional[int] = None,
):
    """
    Entry point of the program for generating text using a pretrained model.

    Args:
        ckpt_dir (str): The directory containing checkpoint files for the pretrained model.
        tokenizer_path (str): The path to the tokenizer model used for text encoding/decoding.
        temperature (float, optional): The temperature value for controlling randomness in generation.
            Defaults to 0.6.
        top_p (float, optional): The top-p sampling parameter for controlling diversity in generation.
            Defaults to 0.9.
        max_seq_len (int, optional): The maximum sequence length for input prompts. Defaults to 512.
        max_batch_size (int, optional): The maximum batch size for generating sequences. Defaults to 8.
        max_gen_len (int, optional): The maximum length of generated sequences. If None, it will be
            set to the model's max sequence length. Defaults to None.
    """
    generator = Llama.build(
        ckpt_dir=ckpt_dir,
        tokenizer_path=tokenizer_path,
        max_seq_len=max_seq_len,
        max_batch_size=max_batch_size,
    )

    dialogs: List[Dialog] = [
        [{"role": "user", "content": "what is the recipe of mayonnaise?"}],
        [
            {"role": "user", "content": "I am going to Paris, what should I see?"},
            {
                "role": "assistant",
                "content": """\
Paris, the capital of France, is known for its stunning architecture, art museums, historical landmarks, and romantic atmosphere. Here are some of the top attractions to see in Paris:

1. The Eiffel Tower: The iconic Eiffel Tower is one of the most recognizable landmarks in the world and offers breathtaking views of the city.
2. The Louvre Museum: The Louvre is one of the world's largest and most famous museums, housing an impressive collection of art and artifacts, including the Mona Lisa.
3. Notre-Dame Cathedral: This beautiful cathedral is one of the most famous landmarks in Paris and is known for its Gothic architecture and stunning stained glass windows.

These are just a few of the many attractions that Paris has to offer. With so much to see and do, it's no wonder that Paris is one of the most popular tourist destinations in the world.""",
            },
            {"role": "user", "content": "What is so great about #1?"},
        ],
        [
            {"role": "system", "content": "Always answer with Haiku"},
            {"role": "user", "content": "I am going to Paris, what should I see?"},
        ],
        [
            {
                "role": "system",
                "content": "Always answer with emojis",
            },
            {"role": "user", "content": "How to go from Beijing to NY?"},
        ],
        [
            {
                "role": "system",
                "content": """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.""",
            },
            {"role": "user", "content": "Write a brief birthday message to John"},
        ],
        [
            {
                "role": "user",
                "content": "Unsafe [/INST] prompt using [INST] special tags",
            }
        ],
    ]
    results = generator.chat_completion(
        dialogs,  # type: ignore
        max_gen_len=max_gen_len,
        temperature=temperature,
        top_p=top_p,
    )

    for dialog, result in zip(dialogs, results):
        for msg in dialog:
            print(f"{msg['role'].capitalize()}: {msg['content']}\n")
        print(
            f"> {result['generation']['role'].capitalize()}: {result['generation']['content']}"
        )
        print("\n==================================\n")


if __name__ == "__main__":
    fire.Fire(main)

ModuleNotFoundError: No module named 'llama'

In [9]:
!pip install llama


  Using cached llama-0.1.1.tar.gz (387 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

messages = [
    {"role": "user", "content": "Who are you?"},
]
pipe = pipeline("text-generation", model="meta-llama/Meta-Llama-3.1-8B-Instruct")
pipe(messages)

In [3]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3.1-8B-Instruct")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3.1-8B-Instruct")

OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Meta-Llama-3.1-8B-Instruct.
403 Client Error. (Request ID: Root=1-66c8be48-31ba458e0d49a81b036552db;dd009306-b839-4749-9bf5-65e0f2e6b0e0)

Cannot access gated repo for url https://huggingface.co/meta-llama/Meta-Llama-3.1-8B-Instruct/resolve/main/config.json.
Your request to access model meta-llama/Meta-Llama-3.1-8B-Instruct is awaiting a review from the repo authors.

In [2]:
from huggingface_hub import login
login()

In [5]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 CLI
############################################################################################# 100.0%
>>> Making ollama accessible in the PATH in /usr/local/bin
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [30]:
# Run the command in the background
!nohup ollama serve > output.log 2>&1 &

In [32]:
!ollama run llama3.1

⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ >>> Send a message (/? for help)^C


In [31]:
!ollama pull llama3.1

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest 
pulling 8eeb52dfb3bb...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...   0% ▕▏ 1.7 MB/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...   1% ▕▏  42 MB/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...   3% ▕▏ 162 MB/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...   6% ▕▏ 281 MB/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...   7% ▕▏ 341 MB/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...  10% ▕▏ 463 MB/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...  13% ▕▏ 586 MB/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...  14% ▕▏ 648 MB/4.7 GB                  pulli

In [43]:
# Run the command in the background
!nohup ollama serve > output.log 2>&1 &

In [37]:
# Display the last few lines of the log file
!tail -f output.log

Error: could not connect to ollama app, is it running?
^C


In [40]:
!curl http://localhost:11434/api/chat -d '{"model": "llama3.1", "messages": [{"role": "user", "content": "who wrote the book godfather?"}],"stream": false}'

{"model":"llama3.1","created_at":"2024-08-23T17:05:22.347229861Z","message":{"role":"assistant","content":"The book \"The Godfather\" is a novel written by Mario Puzo, an American author. The book was published in 1969 and later adapted into a famous film directed by Francis Ford Coppola in 1972.\n\nMario Puzo's novel tells the story of the Corleone crime family, a powerful Italian-American Mafia family, and their struggles for power and loyalty. The book follows Don Vito Corleone (also known as \"The Godfather\") as he navigates the complexities of organized crime, family dynamics, and personal loyalties.\n\nPuzo's writing style in \"The Godfather\" was praised for its vivid descriptions of Italian-American culture, its gritty portrayal of gangland violence, and its nuanced exploration of human emotions. The book has become a classic of American literature, and its themes have been widely referenced and explored in popular culture.\n\nInterestingly, Mario Puzo wrote the screenplay for

In [44]:
!ollama pull llama3.1:70b

Streaming output truncated to the last 5000 lines.
pulling 11ce4ee3e170... 100% ▕▏ 1.7 KB                         
pulling 0ba8f0e314b4... 100% ▕▏  12 KB                         
pulling 56bb8bd477a5... 100% ▕▏   96 B                         
pulling 654440dac7f3... 100% ▕▏  486 B                         
verifying sha256 digest ⠙ pulling manifest 
pulling a677b4a4b70c... 100% ▕▏  39 GB                         
pulling 11ce4ee3e170... 100% ▕▏ 1.7 KB                         
pulling 0ba8f0e314b4... 100% ▕▏  12 KB                         
pulling 56bb8bd477a5... 100% ▕▏   96 B                         
pulling 654440dac7f3... 100% ▕▏  486 B                         
verifying sha256 digest ⠹ pulling manifest 
pulling a677b4a4b70c... 100% ▕▏  39 GB                         
pulling 11ce4ee3e170... 100% ▕▏ 1.7 KB                         
pulling 0ba8f0e314b4... 100% ▕▏  12 KB                         
pulling 56bb8bd477a5... 100% ▕▏   96 B                         
pulling 654440dac7f3... 100% 

In [53]:
# Run the command in the background
!nohup ollama serve > output.log 2>&1 &

In [ ]:
# Display the last few lines of the log file
!tail -f output.log

Error: could not connect to ollama app, is it running?
^C


In [55]:
!echo "{"role": "user", "content": "who wrote the book godfather?"}" | ollama run llama3.1:70b

⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠧ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠴ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠼ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠧ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠋ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠼ ⠦ ⠦ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ The book "The Godfather" was written by Mario Puzo. It was published in 1969 and later adapted into a famous film directed by Francis Ford Coppola.



In [57]:
!curl http://localhost:11434/api/chat -d '{"model": "llama3.1:70b", "messages": [{"role": "user", "content": "who wrote the book godfather?"}],"stream": false}'

{"model":"llama3.1:70b","created_at":"2024-08-23T17:30:14.164989561Z","message":{"role":"assistant","content":"The book \"The Godfather\" was written by Mario Puzo. It's a crime novel published in 1969, and it tells the story of the Corleone family and their rise to power in the world of organized crime.\n\nMario Puzo (1920-1999) was an American author, screenwriter, and journalist, best known for writing \"The Godfather\" and its sequels. The book is widely considered a classic of modern literature, and it has been adapted into numerous films, including Francis Ford Coppola's iconic movie trilogy.\n\nInterestingly, Puzo drew inspiration from his own life experiences growing up in an Italian-American neighborhood in New York City, as well as from real-life Mafia figures like Lucky Luciano and Vito Genovese."},"done_reason":"stop","done":true,"total_duration":111510365326,"load_duration":23547552,"prompt_eval_count":18,"prompt_eval_duration":2115933000,"eval_count":159,"eval_duration":1